In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [13]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Piauí - Produção de Cimento (t)', axis=1)
data

,Ano,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),...,Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Consumo de Cimento (t)
0,2008,0.636377,7.951466,0.633132,0.744861,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,5.483335e+08,...,11.760983,0.670241,24.296582,1002.253734,1279.489487,28.374914,21.343931,67.378713,1.314599,449.076000
1,2009,0.638637,7.981121,0.637582,0.746332,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,5.302764e+08,...,9.472585,0.672829,23.274048,1002.162518,1397.889138,27.971629,21.269184,69.377054,1.242652,536.766000
2,2010,0.640114,7.946167,0.641655,0.747336,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,5.046425e+08,...,9.332485,0.674904,19.182853,1002.283003,1437.681759,27.813041,21.237580,70.062576,1.227247,639.535000
3,2011,0.640823,7.934312,0.645347,0.747880,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,4.854622e+08,...,10.985689,0.676479,18.107389,1002.395371,1443.014477,27.920185,21.009865,69.079510,1.247622,625.022000
4,2012,0.640158,9.098665,0.648074,0.747956,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,4.867661e+08,...,8.159014,0.677105,18.252844,1002.464093,1406.026571,27.983708,20.651339,67.998886,1.231493,687.536000
5,2013,0.640588,9.874811,0.652080,0.747535,2.552759e+07,1.456057e+06,6.451661,2.003921e+07,5.225848e+08,...,7.913561,0.678531,18.670752,1002.768122,1348.345803,27.951493,20.492168,67.705538,1.231771,778.779000
6,2014,0.638470,10.508945,0.654503,0.746830,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,5.676413e+08,...,10.350442,0.678492,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497,755.713857
7,2015,0.636259,10.965401,0.655795,0.745732,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,5.179392e+08,...,12.454398,0.677866,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249,794.945486
8,2016,0.648680,11.203809,0.670587,0.753641,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,4.524144e+08,...,13.111908,0.689454,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215,842.687657
9,2017,0.648105,11.173036,0.673185,0.752518,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,4.213689e+08,...,9.479812,0.689859,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392,831.356000


In [14]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.636377,7.951466,0.633132,0.744861,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,5.483335e+08,2.546644e+09,...,2.868108,11.760983,0.670241,24.296582,1002.253734,1279.489487,28.374914,21.343931,67.378713,1.314599
1,0.638637,7.981121,0.637582,0.746332,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,5.302764e+08,2.679945e+09,...,2.305885,9.472585,0.672829,23.274048,1002.162518,1397.889138,27.971629,21.269184,69.377054,1.242652
2,0.640114,7.946167,0.641655,0.747336,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,5.046425e+08,2.891930e+09,...,1.958246,9.332485,0.674904,19.182853,1002.283003,1437.681759,27.813041,21.237580,70.062576,1.227247
3,0.640823,7.934312,0.645347,0.747880,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,4.854622e+08,3.091988e+09,...,2.079088,10.985689,0.676479,18.107389,1002.395371,1443.014477,27.920185,21.009865,69.079510,1.247622
4,0.640158,9.098665,0.648074,0.747956,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,4.867661e+08,3.189506e+09,...,2.183282,8.159014,0.677105,18.252844,1002.464093,1406.026571,27.983708,20.651339,67.998886,1.231493
5,0.640588,9.874811,0.652080,0.747535,2.552759e+07,1.456057e+06,6.451661,2.003921e+07,5.225848e+08,3.093872e+09,...,2.403445,7.913561,0.678531,18.670752,1002.768122,1348.345803,27.951493,20.492168,67.705538,1.231771
6,0.638470,10.508945,0.654503,0.746830,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,5.676413e+08,2.779340e+09,...,2.748397,10.350442,0.678492,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497
7,0.636259,10.965401,0.655795,0.745732,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,5.179392e+08,2.561731e+09,...,2.501529,12.454398,0.677866,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249
8,0.648680,11.203809,0.670587,0.753641,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,4.524144e+08,2.516859e+09,...,2.073464,13.111908,0.689454,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215
9,0.648105,11.173036,0.673185,0.752518,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,4.213689e+08,2.497710e+09,...,2.177386,9.479812,0.689859,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392


In [15]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     536.766000
1     639.535000
2     625.022000
3     687.536000
4     778.779000
5     755.713857
6     794.945486
7     842.687657
8     831.356000
9     626.223000
10    702.437000
11    788.164000
12    826.368000
13    804.417000
14           NaN
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [16]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.244793,-1.286179,-1.543242,-1.364689,-1.791927,-2.430395,-1.983161,-1.934577,1.204752,-0.990234,...,2.026352,0.984445,-1.486495,1.532542,-1.329511,-0.499138,1.719528,1.574885,0.008389,2.526713
1,-0.733935,-1.264664,-1.229292,-0.796636,-1.615264,-1.276024,-1.702934,-1.710653,0.820657,-0.546688,...,0.085558,-0.051710,-1.104860,1.094793,-1.637011,0.971937,-0.703092,1.403767,1.371924,-0.247365
2,-0.399968,-1.290024,-0.941897,-0.409264,-1.098406,0.191239,-1.000708,-1.029511,0.275392,0.158672,...,-1.114490,-0.115145,-0.798779,-0.656658,-1.230840,1.466347,-1.655760,1.331414,1.839678,-0.841315
3,-0.239582,-1.298625,-0.681446,-0.199210,-0.616760,0.654083,-0.420225,-0.457837,-0.132596,0.824346,...,-0.697344,0.633403,-0.566581,-1.117067,-0.852032,1.532604,-1.012123,0.810102,1.168900,-0.055737
4,-0.389944,-0.453869,-0.489065,-0.170018,-0.190143,0.804775,-0.016723,-0.032922,-0.104860,1.148829,...,-0.337667,-0.646476,-0.474249,-1.054798,-0.620359,1.073042,-0.630530,-0.010676,0.431554,-0.677618
5,-0.292755,0.109237,-0.206415,-0.332297,0.201584,1.133256,0.361197,0.354549,0.657046,0.830615,...,0.422337,-0.757614,-0.264062,-0.875890,0.404563,0.356378,-0.824051,-0.375069,0.231393,-0.666887
6,-0.771591,0.569311,-0.035440,-0.604609,0.549114,1.310516,0.710469,0.697572,1.615453,-0.215960,...,1.613108,0.345772,-0.269828,-1.005951,0.950789,-0.092851,0.075353,-0.946654,-0.590506,0.556362
7,-1.271487,0.900476,0.055711,-1.028293,0.752766,0.327112,0.717082,0.733641,0.558230,-0.940033,...,0.760923,1.298413,-0.362089,-0.659624,0.817133,-0.716594,0.718932,-1.401597,-1.254312,1.048065
8,1.536902,1.073445,1.099309,2.024843,0.920581,-0.313247,0.813122,0.834017,-0.835563,-1.089341,...,-0.716758,1.596124,1.346608,0.608069,0.621045,-1.049353,0.536936,-1.282101,-1.340487,0.699728
9,1.406981,1.051119,1.282663,1.591118,1.025520,-0.308314,0.960590,0.965973,-1.495939,-1.153056,...,-0.358019,-0.048438,1.406313,1.519016,0.916592,-1.310878,-0.121575,-0.485063,-0.560729,-0.797166


In [17]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     536.766000
1     639.535000
2     625.022000
3     687.536000
4     778.779000
5     755.713857
6     794.945486
7     842.687657
8     831.356000
9     626.223000
10    702.437000
11    788.164000
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [18]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.823567,0.785725,1.126392,-0.598225,0.529639,0.378729,0.253155,0.276615,1.108405,2.635823,...,-1.482779,-2.260447,0.91739,1.011919,0.418013,-0.843464,1.091606,-0.183281,-0.442768,-1.555022


In [19]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    826.368
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [20]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [21]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [23]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[305215137, 1911003474, 3587820505, 1692861923, 2341864209, 2428814742, 2839185617, 3316154237, 173475192, 3921559424, 768010267, 3282428241, 3275898227, 1653462929, 240638560, 1093551399, 3415400006, 1637282662, 63965357, 108531822, 3208331658, 4272948104, 2176551367, 2360968497, 1779784768, 3713503335, 2518023779, 138741936, 1704763447, 3172572954, 2705360656, 1655508184, 3112687030, 2217987431, 623523280, 1217457592, 2532375125, 1584360726, 2014066680, 2775435308, 1762025197, 174690573, 3072026732, 3762957063, 2895440402, 1382559772, 3230651107, 1552666952, 675150867, 900758356, 1456208840, 1492467137, 541796833, 3921699460, 3474798200, 3655814160, 2440048257, 907735530, 2426880467, 16672622, 1669112055, 3708229240, 3117547371, 927197053, 4239897826, 3839297399, 3647483866, 3477269925, 3077382904, 2118857694, 1305133596, 1583172625, 3429266692, 3093113115, 3508115611, 2517693496, 740106325, 469964152, 1648040638, 3633661070, 2443158889, 4188179312, 3530846744, 2770703230, 1526460343

val_loss: 2477.698974609375


Step: 84 ___________________________________________
val_loss: 985.9883422851562


Step: 85 ___________________________________________
val_loss: 762.1360473632812


Step: 86 ___________________________________________
val_loss: 229.94139099121094


Step: 87 ___________________________________________
val_loss: 212.5199432373047


Step: 88 ___________________________________________
val_loss: 73.45130157470703


Step: 89 ___________________________________________
val_loss: 50.6439323425293


Step: 90 ___________________________________________
val_loss: 1169.4154052734375


Step: 91 ___________________________________________
val_loss: 458.99871826171875


Step: 92 ___________________________________________
val_loss: 817.9620971679688


Step: 93 ___________________________________________
val_loss: 88.52117919921875


Step: 94 ___________________________________________
val_loss: 109.36737060546875


Step: 95 ___________________________________________
v

In [24]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 454ms/step - loss: 523390.1562 - val_loss: 625571.0000
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 509542.0938 - val_loss: 383256.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 482278.6562 - val_loss: 899257.3750
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 504962.5938 - val_loss: 395966.3125
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 487060.3750 - val_loss: 563719.8750
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 474495.5938 - val_loss: 461306.6875
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 463198.4062 - val_loss: 478241.7188
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 453982.6250 - val_loss: 471531.8438
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 445071.2500 - val_loss: 461784.8750
Epoch 10/

Epoch 75/10000
1/1 [==============================] - 0s 16ms/step - loss: 124875.9922 - val_loss: 131118.4219
Epoch 76/10000
1/1 [==============================] - 0s 16ms/step - loss: 119366.1875 - val_loss: 104660.3047
Epoch 77/10000
1/1 [==============================] - 0s 16ms/step - loss: 118611.1953 - val_loss: 133147.7031
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 114622.0234 - val_loss: 118965.5625
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 106930.0703 - val_loss: 111269.5781
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 108132.5625 - val_loss: 108830.3984
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 111079.1328 - val_loss: 100621.6172
Epoch 82/10000
1/1 [==============================] - 0s 15ms/step - loss: 107754.6875 - val_loss: 136560.6562
Epoch 83/10000
1/1 [==============================] - 0s 16ms/step - loss: 101779.7109 - val_loss: 128831.7656
E

1/1 [==============================] - 0s 15ms/step - loss: 32502.2812 - val_loss: 42604.1719
Epoch 150/10000
1/1 [==============================] - 0s 15ms/step - loss: 32743.3828 - val_loss: 41547.5469
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 33159.7656 - val_loss: 38682.2031
Epoch 152/10000
1/1 [==============================] - 0s 15ms/step - loss: 32539.1660 - val_loss: 46022.2969
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 32302.2520 - val_loss: 41509.8359
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 31702.3184 - val_loss: 41261.1523
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 31168.0723 - val_loss: 39860.5938
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 30649.1152 - val_loss: 38447.2305
Epoch 157/10000
1/1 [==============================] - 0s 15ms/step - loss: 29528.6348 - val_loss: 39973.1016
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 13288.3750 - val_loss: 13565.8516
Epoch 224/10000
1/1 [==============================] - 0s 15ms/step - loss: 12917.7705 - val_loss: 13051.6680
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 12810.0898 - val_loss: 13365.4004
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 12773.9717 - val_loss: 13434.1035
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 12636.7344 - val_loss: 13671.6670
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 12318.5381 - val_loss: 13134.7490
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 12211.0615 - val_loss: 13398.4443
Epoch 230/10000
1/1 [==============================] - 0s 15ms/step - loss: 12066.4580 - val_loss: 13368.2236
Epoch 231/10000
1/1 [==============================] - 0s 15ms/step - loss: 11925.9170 - val_loss: 13382.7930
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 6747.1484 - val_loss: 10630.1709
Epoch 299/10000
1/1 [==============================] - 0s 16ms/step - loss: 6704.4492 - val_loss: 10672.7158
Epoch 300/10000
1/1 [==============================] - 0s 15ms/step - loss: 6668.2397 - val_loss: 10678.3408
Epoch 301/10000
1/1 [==============================] - 0s 16ms/step - loss: 6622.8247 - val_loss: 10673.2080
Epoch 302/10000
1/1 [==============================] - 0s 16ms/step - loss: 6582.8203 - val_loss: 10687.2363
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 6543.6147 - val_loss: 10692.5498
Epoch 304/10000
1/1 [==============================] - 0s 15ms/step - loss: 6377.6777 - val_loss: 10305.2607
Epoch 305/10000
1/1 [==============================] - 0s 16ms/step - loss: 6471.1660 - val_loss: 10624.3828
Epoch 306/10000
1/1 [==============================] - 0s 15ms/step - loss: 6432.2183 - val_loss: 10572.0293
Epoch 307/10000
1/1 [==============

...

1/1 [==============================] - 0s 18ms/step - loss: 6187.3872 - val_loss: 683.7919
Epoch 2110/10000
1/1 [==============================] - 0s 16ms/step - loss: 6187.1538 - val_loss: 685.4943
Epoch 2111/10000
1/1 [==============================] - 0s 17ms/step - loss: 6186.9253 - val_loss: 687.1860
Epoch 2112/10000
1/1 [==============================] - 0s 16ms/step - loss: 6186.7012 - val_loss: 688.8607
Epoch 2113/10000
1/1 [==============================] - 0s 16ms/step - loss: 6186.4819 - val_loss: 690.5276
Epoch 2114/10000
1/1 [==============================] - 0s 16ms/step - loss: 6186.2676 - val_loss: 692.1807
Epoch 2115/10000
1/1 [==============================] - 0s 16ms/step - loss: 6186.0571 - val_loss: 693.8195
Epoch 2116/10000
1/1 [==============================] - 0s 16ms/step - loss: 6185.8521 - val_loss: 695.4507
Epoch 2117/10000
1/1 [==============================] - 0s 16ms/step - loss: 6185.6509 - val_loss: 697.0612
Epoch 2118/10000
1/1 [=======================

1/1 [==============================] - 0s 16ms/step - loss: 6178.3188 - val_loss: 781.8174
Epoch 2186/10000
1/1 [==============================] - 0s 16ms/step - loss: 6178.2671 - val_loss: 782.7563
Epoch 2187/10000
1/1 [==============================] - 0s 16ms/step - loss: 6178.2163 - val_loss: 783.6856
Epoch 2188/10000
1/1 [==============================] - 0s 16ms/step - loss: 6178.1655 - val_loss: 784.6085
Epoch 2189/10000
1/1 [==============================] - 0s 17ms/step - loss: 6178.1167 - val_loss: 785.5251
Epoch 2190/10000
1/1 [==============================] - 0s 16ms/step - loss: 6178.0684 - val_loss: 786.4321
Epoch 2191/10000
1/1 [==============================] - 0s 16ms/step - loss: 6178.0210 - val_loss: 787.3326
Epoch 2192/10000
1/1 [==============================] - 0s 16ms/step - loss: 6177.9741 - val_loss: 788.2269
Epoch 2193/10000
1/1 [==============================] - 0s 17ms/step - loss: 6177.9282 - val_loss: 789.1182
Epoch 2194/10000
1/1 [=======================

1/1 [==============================] - 0s 16ms/step - loss: 6176.1343 - val_loss: 835.0103
Epoch 2262/10000
1/1 [==============================] - 0s 16ms/step - loss: 6176.1196 - val_loss: 835.5254
Epoch 2263/10000
1/1 [==============================] - 0s 16ms/step - loss: 6176.1060 - val_loss: 836.0266
Epoch 2264/10000
1/1 [==============================] - 0s 16ms/step - loss: 6176.0923 - val_loss: 836.5384
Epoch 2265/10000
1/1 [==============================] - 0s 16ms/step - loss: 6176.0786 - val_loss: 837.0292
Epoch 2266/10000
1/1 [==============================] - 0s 16ms/step - loss: 6176.0659 - val_loss: 837.5273
Epoch 2267/10000
1/1 [==============================] - 0s 16ms/step - loss: 6176.0527 - val_loss: 838.0114
Epoch 2268/10000
1/1 [==============================] - 0s 17ms/step - loss: 6176.0405 - val_loss: 838.4991
Epoch 2269/10000
1/1 [==============================] - 0s 17ms/step - loss: 6176.0273 - val_loss: 838.9834
Epoch 2270/10000
1/1 [=======================

1/1 [==============================] - 0s 17ms/step - loss: 6175.5132 - val_loss: 864.3112
Epoch 2338/10000
1/1 [==============================] - 0s 17ms/step - loss: 6175.5093 - val_loss: 864.5875
Epoch 2339/10000
1/1 [==============================] - 0s 16ms/step - loss: 6175.5044 - val_loss: 864.8712
Epoch 2340/10000
1/1 [==============================] - 0s 17ms/step - loss: 6175.5005 - val_loss: 865.1404
Epoch 2341/10000
1/1 [==============================] - 0s 17ms/step - loss: 6175.4966 - val_loss: 865.4169
Epoch 2342/10000
1/1 [==============================] - 0s 17ms/step - loss: 6175.4927 - val_loss: 865.6898
Epoch 2343/10000
1/1 [==============================] - 0s 17ms/step - loss: 6175.4888 - val_loss: 865.9556
Epoch 2344/10000
1/1 [==============================] - 0s 16ms/step - loss: 6175.4858 - val_loss: 866.2250
Epoch 2345/10000
1/1 [==============================] - 0s 17ms/step - loss: 6175.4819 - val_loss: 866.4910
Epoch 2346/10000
1/1 [=======================

1/1 [==============================] - 0s 15ms/step - loss: 6175.3003 - val_loss: 888.2794
Epoch 2414/10000
1/1 [==============================] - 0s 15ms/step - loss: 6175.2988 - val_loss: 888.3594
Epoch 2415/10000
1/1 [==============================] - 0s 14ms/step - loss: 6175.2983 - val_loss: 888.4359
Epoch 2416/10000
1/1 [==============================] - 0s 14ms/step - loss: 6175.2969 - val_loss: 888.5159
Epoch 2417/10000
1/1 [==============================] - 0s 14ms/step - loss: 6175.2964 - val_loss: 888.5959
Epoch 2418/10000
1/1 [==============================] - 0s 14ms/step - loss: 5633.3188 - val_loss: 558.4724
Epoch 2419/10000
1/1 [==============================] - 0s 17ms/step - loss: 6213.5938 - val_loss: 992.4999
Epoch 2420/10000
1/1 [==============================] - 0s 15ms/step - loss: 6177.9897 - val_loss: 865.4097
Epoch 2421/10000
1/1 [==============================] - 0s 15ms/step - loss: 6175.4849 - val_loss: 898.3235
Epoch 2422/10000
1/1 [=======================

In [25]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 53ms/step
[826.368](test_target) - [[788.2711]](prediction) = 38.09688183593755


In [26]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [27]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.300755,-0.527817,-1.242369,-1.294258,-0.985718,-1.173238,-1.018535,-0.977767,1.148205,-1.122043,...,1.308949,1.412346,-1.267049,0.925774,0.402439,-1.372251,1.360201,1.353721,-1.370823,1.394785
1,0.169724,1.400155,0.036043,0.153497,-0.385361,-0.097218,-0.340403,-0.395977,0.140893,-0.184474,...,-0.190804,-0.643250,0.089521,0.462967,-1.375329,0.389992,-0.344854,-0.322487,0.384358,-0.495076
2,1.131031,-0.872338,1.206325,1.140761,1.371079,1.270457,1.358938,1.373744,-1.289098,1.306518,...,-1.118144,-0.769096,1.177528,-1.388742,0.972890,0.982259,-1.015347,-1.031233,0.986465,-0.899709


train_target:


,Piauí - Consumo de Cimento (t)
0,536.766
1,639.535
2,625.022


test_input:


...

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.823567,0.785725,1.126392,-0.598225,0.529639,0.378729,0.253155,0.276615,1.108405,2.635823,...,-1.482779,-2.260447,0.91739,1.011919,0.418013,-0.843464,1.091606,-0.183281,-0.442768,-1.555022


test_target:


,Piauí - Consumo de Cimento (t)
12,826.368


1/1 [==============================] - 0s 40ms/step
Error: 38.09688183593755




[624.5462036132812,
 657.363525390625,
 744.810546875,
 694.8681640625,
 781.9334106445312,
 819.4349365234375,
 831.3497924804688,
 704.60986328125,
 724.8702392578125,
 788.2711181640625]

In [28]:
display(targets)
display(predictions)

[687.536,
 778.779,
 755.7138571428571,
 794.9454857142857,
 842.6876571428572,
 831.356,
 626.223,
 702.437,
 788.164,
 826.368]

[624.5462036132812,
 657.363525390625,
 744.810546875,
 694.8681640625,
 781.9334106445312,
 819.4349365234375,
 831.3497924804688,
 704.60986328125,
 724.8702392578125,
 788.2711181640625]

In [29]:
mae = mean_absolute_error(predictions, targets)
mae

67.67515112304689

In [30]:
porcentage = mae/np.mean(targets)
porcentage

0.0886472223360988